In [1]:
%load_ext autoreload
%autoreload 2
import os
if not os.path.exists('./modules') and not os.path.exists('modules.zip'):
    from google.colab import files
    uploaded = files.upload()
if not os.path.exists('./modules') and os.path.exists('modules.zip'):
    os.system('unzip modules.zip -d .')

!pip3 install optuna
import kagglehub
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import TensorDataset, DataLoader
import optuna
from modules.external_dataset import EEGDataset
from modules.utils import split_and_get_loaders, evaluate_model, manual_write_study_params
import torch.nn.functional as F
import matplotlib.pyplot as plt
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Saving modules.zip to modules.zip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 23.5 MB/s eta 0:00:00


device(type='cuda')

In [2]:
#! need to modify those for the competition itself
TRIAL_LENGTH = 640  # frequency of changing.. frequency
# Download dataset
path_1 = kagglehub.dataset_download("girgismicheal/steadystate-visual-evoked-potential-signals")
path_1 += "/SSVEP (BrainWheel)"
print("Download datasetaset files:", "\n", path_1)

Download datasetaset files: 
 /kaggle/input/steadystate-visual-evoked-potential-signals/SSVEP (BrainWheel)


In [8]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model) # Batch-first for consistency with transformer
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [batch_size, seq_len, embedding_dim]
        """
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

class TransformerHead(nn.Module):
    def __init__(self, input_dim, nhead, num_encoder_layers, dim_feedforward, output_dim, dropout=0.1, max_sequence_length=512):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers)
        self.fc = nn.Linear(input_dim, output_dim) # Output layer

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [batch_size, seq_len, feature_dim] (which is F2 from EEGNet) Should apply positional encoding
        """
        output = self.transformer_encoder(x)
        output = self.fc(output[:, -1, :])
        return output

class DepthWiseConv2D(nn.Module):
    def __init__(self, in_channels, kernel_size, dim_mult=1, padding=0, bias=False):
        super(DepthWiseConv2D, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels * dim_mult, padding=padding, kernel_size=kernel_size, groups=in_channels, bias=bias)

    def forward(self, x: torch.Tensor):
        return self.depthwise(x)


class SeperableConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, bias=False):
        super(SeperableConv2D, self).__init__()
        self.depthwise = DepthWiseConv2D(in_channels, kernel_size, padding=padding)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out



class SSVEPClassifier(nn.Module):
    def __init__(self, n_electrodes=16, n_samples=128, out_dim=4, dropout=0.25, kernLength=256, F1=96, D=1, F2=96,
                 nhead=8, num_encoder_layers=2, dim_feedforward=2048,
                 max_sequence_length=None): # This should be at least n_samples // (2 * 4) = n_samples // 8
        super().__init__()
        max_sequence_length = max_sequence_length or n_samples // 8

        # B x C x T
        self.block_1 = nn.Sequential(
            nn.Conv2d(1, F1, (1, kernLength), padding='same', bias=False),
            nn.BatchNorm2d(F1),
            #
            DepthWiseConv2D(F1, (n_electrodes, 1), dim_mult=D, bias=False),
            nn.BatchNorm2d(F1*D),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(dropout),
            #
            SeperableConv2D(F1 * D, F2, kernel_size=(1, 16), padding='same', bias=False),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.MaxPool2d((1, 4)),
            nn.Dropout(dropout),
        )

        self.transformer_head = TransformerHead(
            input_dim=F2,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            dim_feedforward=dim_feedforward,
            output_dim=out_dim,
            dropout=dropout,
            max_sequence_length=max_sequence_length
        )

    def forward(self, x: torch.Tensor):
        """expected input shape: BxCxT"""
        x = x.unsqueeze(1)
        y = self.block_1(x)

        y = y.squeeze(2)
        y = y.permute(0, 2, 1) # B x (T / 8) x F2

        y = self.transformer_head(y)

        return y


dummy_x = torch.randn(5, 14, 320)
model = SSVEPClassifier(n_electrodes=dummy_x.shape[1], n_samples=dummy_x.shape[2])
model(dummy_x)

tensor([[-0.3402,  1.0259,  0.3528, -0.1217],
        [-0.8049,  0.5517,  0.4551,  0.3997],
        [ 0.0154,  0.7810, -0.2736,  0.7535],
        [-0.3659,  0.6799,  0.4175,  0.3663],
        [-0.4647,  0.2742,  0.1245, -0.2464]], grad_fn=<AddmmBackward0>)

In [21]:
class Trainer:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.train_epochs = 1000
        self.tune_epochs = 50
        self.optuna_n_trials = 120

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = None
        self.trial = None

        self.train_loader = None
        self.eval_loader = None
        self.test_loader = None
        self.dataset = None

        self.storage = "sqlite:///optuna_studies.db"
        self.study_name = "ssvep_classifier_optimization"

        self.checkpoint_path = "./checkpoints/ssvep"
        os.makedirs(os.path.join(self.checkpoint_path, "models"), exist_ok=True)
        self.checkpoint_model_path = os.path.join(self.checkpoint_path, "models")

    def _train_loop(self, n_epochs: int, should_save=False, should_print=False):
        assert isinstance(self.optimizer, torch.optim.Optimizer), "optimizer is not a valid optimizer"
        assert isinstance(self.train_loader, DataLoader), "train_laoder is not valid Datloader"
        if self.trial is None:
            print("Warning: self.trial is none, we are probably in acutal training phase")

        for epoch in range(n_epochs):
            self.model.to(self.device)
            self.model.train()

            avg_loss = 0
            for x, y in self.train_loader:
                x = x.to(self.device)
                y = y.to(self.device)

                y_pred = self.model(x)  # B x out_size
                loss = self.criterion(y_pred, y)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                avg_loss += loss.item()

            avg_loss = avg_loss / len(self.train_loader)
            evaluation = evaluate_model(self.model, self.val_loader, self.device)

            if self.trial is not None:
                self.trial.report(evaluation, epoch)
                if self.trial.should_prune():
                    optuna.exceptions.TrialPruned()

            if should_print:
                print(f"epoch {epoch}, evaluation {evaluation}, avg_loss {avg_loss}")

            if should_save:
                self.model.cpu()
                torch.save(self.model.state_dict(), os.path.join(self.checkpoint_model_path, f"ssvep.pth"))
                self.model.to(self.device)

    def _prepare_training(self, is_trial, my_lr=None):
        if is_trial:
            assert isinstance(self.trial, optuna.Trial), "trial is none, cant' suggest params"

            # EEGNet network params
            dropout = self.trial.suggest_float("dropout", 0, 0.5)
            kernLength = self.trial.suggest_categorical("kernLength", [128, 256, 512])
            F1 = self.trial.suggest_categorical("F1", [64, 96, 128])
            D = self.trial.suggest_categorical("D", [1, 2, 3])
            F2 = self.trial.suggest_categorical("F2", [64, 96, 128]) # F2 is input_dim for transformer, so keep it as categorical

            # Transformer-specific network params
            nhead = self.trial.suggest_categorical("nhead", [4, 8, 16]) # Should divide F2
            num_encoder_layers = self.trial.suggest_int("num_encoder_layers", 1, 4)
            dim_feedforward = self.trial.suggest_categorical("dim_feedforward", [256, 512, 1024])

            window_length = self.trial.suggest_categorical("window_length", [160, 320, 640])

            # training params
            lr = self.trial.suggest_float("lr", 3e-4, 3e-2, log=True)
            batch_size = self.trial.suggest_categorical("batch_size", [32, 64])

        else:
            best_params = self._get_study().best_params

            window_length = best_params['window_length']
            dropout = best_params["dropout"]
            lr = best_params["lr"]
            batch_size = best_params["batch_size"]

            nhead = best_params['nhead']
            num_encoder_layers = best_params["num_encoder_layers"]
            dim_feedforward = best_params["dim_feedforward"]

            kernLength = best_params.get("kernLength", 256)
            F1 = best_params.get("F1", 96)
            D = best_params.get("D", 1)
            F2 = best_params.get("F2", 96)

        lr = my_lr or lr
        stride = int(window_length // 3)
        self.dataset = EEGDataset(path_1, TRIAL_LENGTH, window_length, stride=stride)
        unique_freqs = torch.unique(self.dataset.labels)

        n_samples = self.dataset.data[0].shape[1] # data[x] shape CxT (this is 'window_length' here)
        n_electrodes = self.dataset.data[0].shape[0]
        out_size = len(unique_freqs)
        self.model = SSVEPClassifier(
            n_electrodes=n_electrodes,
            n_samples=n_samples, # This is 'window_length' for the current dataset
            out_dim=out_size,
            dropout=dropout,
            kernLength=kernLength,
            F1=F1,
            D=D,
            F2=F2, # F2 for the transformer's input_dim
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            dim_feedforward=dim_feedforward,
        )

        self.train_loader, self.val_loader, self.test_loader = split_and_get_loaders(self.dataset, batch_size)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)


    def _objective(self, trial: optuna.Trial):
        self.trial = trial
        self._prepare_training(True)

        self._train_loop(self.tune_epochs, should_save=False, should_print=False)
        evaluation = evaluate_model(self.model, self.val_loader, self.device)
        return evaluation

    def _get_study(self):
        return optuna.create_study(study_name=self.study_name, storage=self.storage, direction="maximize", load_if_exists=True)

    def optimize(self, delete_existing=False):
        if delete_existing:
            try:
                optuna.delete_study(study_name=self.study_name, storage=self.storage)
            except Exception:
                pass

        study = self._get_study()
        study.optimize(self._objective, n_trials=self.optuna_n_trials, timeout=60 * 10)

        # Print optimization results
        print("\nStudy statistics:")
        print(f"  Number of finished trials: {len(study.trials)}")
        print(f"  Number of pruned trials: {len(study.get_trials(states=[optuna.trial.TrialState.PRUNED]))}")
        print(f"  Number of complete trials: {len(study.get_trials(states=[optuna.trial.TrialState.COMPLETE]))}")

        print("\nBest trial:")
        trial = study.best_trial
        print(f"  Value: {trial.value}")
        print("\nBest hyperparameters:")
        for key, value in trial.params.items():
            print(f"  {key}: {value}")

        return study.best_params

    def train(self):
        self.trial = None
        self._prepare_training(False)

        self._train_loop(self.train_epochs, should_save=True, should_print=True)
        evaluation = evaluate_model(self.model, self.val_loader, self.device)
        print("done training")
        return evaluation

In [22]:
trainer = Trainer()
delete_existing = False
trainer.optimize(delete_existing)

[I 2025-06-20 01:58:39,489] Using an existing study with name 'ssvep_classifier_optimization' instead of creating a new one.
[W 2025-06-20 01:58:47,375] Trial 7 failed with parameters: {'dropout': 0.45328661235969725, 'kernLength': 512, 'F1': 96, 'D': 3, 'F2': 128, 'nhead': 8, 'num_encoder_layers': 1, 'dim_feedforward': 512, 'window_length': 160, 'lr': 0.003203741430612504, 'batch_size': 32} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-21-1404354344.py", line 133, in _objective
    self._train_loop(self.tune_epochs, should_save=False, should_print=False)
  File "/tmp/ipython-input-21-1404354344.py", line 46, in _train_loop
    avg_loss += loss.item()
                ^^^^^^^^^^^
KeyboardInterrupt
[W 2025-06-20 01:58:47,376] Trial 7 failed with value Non

KeyboardInterrupt: 

In [23]:
# manual_write_study_params(trainer.study_name, trainer.storage)
trainer.train()

[I 2025-06-20 01:58:49,365] Using an existing study with name 'ssvep_classifier_optimization' instead of creating a new one.


epoch 0, evaluation 0.2125, avg_loss 1.4173949718475343
epoch 1, evaluation 0.2375, avg_loss 1.4021853685379029
epoch 2, evaluation 0.26875, avg_loss 1.391485148668289
epoch 3, evaluation 0.30625, avg_loss 1.3985726237297058
epoch 4, evaluation 0.2875, avg_loss 1.3951397955417633
epoch 5, evaluation 0.275, avg_loss 1.3945764303207397
epoch 6, evaluation 0.28125, avg_loss 1.3931984603404999
epoch 7, evaluation 0.275, avg_loss 1.4004414200782775
epoch 8, evaluation 0.2625, avg_loss 1.3852189779281616
epoch 9, evaluation 0.26875, avg_loss 1.3753587126731872
epoch 10, evaluation 0.2625, avg_loss 1.379552412033081
epoch 11, evaluation 0.31875, avg_loss 1.3828181862831115
epoch 12, evaluation 0.30625, avg_loss 1.379877758026123
epoch 13, evaluation 0.29375, avg_loss 1.3731835603713989
epoch 14, evaluation 0.275, avg_loss 1.3852452695369721
epoch 15, evaluation 0.33125, avg_loss 1.3707289338111877
epoch 16, evaluation 0.28125, avg_loss 1.3684512734413148
epoch 17, evaluation 0.2875, avg_loss 

KeyboardInterrupt: 